In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
import time

In [2]:
raw = pd.read_csv('train.csv')

In [3]:
print(raw.info)

<bound method DataFrame.info of          order_detail_id  order_id  order_total_num  order_amount  \
0                1000000   1000000              1.0         239.9   
1                1001530   1001327              2.0         288.0   
2                1001531   1001327              2.0         288.0   
3                1001532   1001328              3.0         180.0   
4                1001533   1001329              1.0         159.9   
...                  ...       ...              ...           ...   
2306866          3685495   3238358              1.0         199.0   
2306867          3685496   3238359              2.0         299.8   
2306868          3685497   3238359              2.0         299.8   
2306869          3685498   3238360              1.0         168.0   
2306870          3685499   3238361              1.0         102.0   

         order_total_payment  order_total_discount       order_pay_time  \
0                       96.9                   0.0  2012-11-01 0

In [4]:
raw['goods_id'] = pd.factorize(raw['goods_id'])[0]

In [5]:
raw.columns

Index(['order_detail_id', 'order_id', 'order_total_num', 'order_amount',
       'order_total_payment', 'order_total_discount', 'order_pay_time',
       'order_status', 'order_count', 'is_customer_rate',
       'order_detail_status', 'order_detail_goods_num', 'order_detail_amount',
       'order_detail_payment', 'order_detail_discount', 'customer_province',
       'customer_city', 'member_id', 'customer_id', 'customer_gender',
       'member_status', 'is_member_actived', 'goods_id', 'goods_class_id',
       'goods_price', 'goods_status', 'goods_has_discount', 'goods_list_time',
       'goods_delist_time'],
      dtype='object')

In [6]:
def prerpocess(raw, train='train'):
    st = time.time()
    data = pd.DataFrame(raw.groupby('customer_id')['customer_gender'].last().fillna(0))
    data[['goods_id_last', 'goods_status_last', 'goods_price_last', 'goods_has_discount_last', 'goods_list_time_last', 'goods_delist_time_last']] = raw.groupby('customer_id')['goods_id', 'goods_status', 'goods_price', 'goods_has_discount', 'goods_list_time', 'goods_delist_time'].last()
    data[['order_total_num_last', 'order_amount_last',
       'order_total_payment_last', 'order_total_discount_last', 'order_pay_time_last',
       'order_status_last', 'order_count_last', 'is_customer_rate_last',
       'order_detail_status_last', 'order_detail_goods_num_last', 'order_detail_amount_last',
       'order_detail_payment_last', 'order_detail_discount_last']] = raw.groupby('customer_id')['order_total_num', 'order_amount',
       'order_total_payment', 'order_total_discount', 'order_pay_time',
       'order_status', 'order_count', 'is_customer_rate',
       'order_detail_status', 'order_detail_goods_num', 'order_detail_amount',
       'order_detail_payment', 'order_detail_discount'].last()
    data[['good_price_std', 'good_price_mean', 'good_price_min', 'good_price_max']] = raw.groupby('customer_id')['goods_price'].agg({'good_price_std':'std', 'good_price_mean':'mean', 'good_price_min':'min', 'good_price_max':'max'})
    data[['order_detail_payment_std', 'order_detail_payment_mean', 'order_detail_payment_min', 'order_detail_payment_max']] = raw.groupby('customer_id')['order_detail_payment'].agg({'order_detail_payment_std':'std', 'order_detail_payment_mean':'mean', 'order_detail_payment_min':'min', 'order_detail_payment_max':'max'})
    data['count'] = raw.groupby('customer_id')['order_id'].nunique()
    data['goods_count'] = raw.groupby('customer_id')['order_total_num'].sum()
    data['customer_province'] = raw.groupby('customer_id')['customer_province'].last()
    data['customer_city'] = raw.groupby('customer_id')['customer_city'].last()
    data[['is_customer_rate_ratio','is_customer_rate_sum']] = raw.groupby('customer_id')['is_customer_rate'].agg({'is_customer_rate_ratio':np.mean,'is_customer_rate_sum':np.sum})
    data['order_detail_count'] = raw.groupby('customer_id')['customer_id'].count()
    data[['goods_has_discount_sum','goods_has_discount_ave']] = raw.groupby('customer_id')['goods_has_discount'].agg({'goods_has_discount_sum':np.sum,'goods_has_discount_ave':np.mean})
    data[['order_total_payment_sum','order_total_ave_pay']] = raw.groupby('customer_id')['order_total_payment'].agg({'order_total_payment_sum':np.sum,'order_total_ave_pay':np.mean})
    data[['order_total_num_sum', 'order_total_num_ave']] = raw.groupby('customer_id')['order_total_num'].agg({'order_total_num_sum':np.sum,'order_total_num_ave':np.mean})

    
    def time2multi(x):
        t=datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')
        return pd.Series([t.month,t.day,t.weekday(),t.hour,t.minute,t.second])
    
    data[['order_pay_time_last_m','order_pay_time_last_d','order_pay_time_last_week','order_pay_time_last_h','order_pay_time_last_min','order_pay_time_last_s']]=data['order_pay_time_last'].apply(time2multi)
    #data[['order_pay_time_last_m','order_pay_time_last_d','order_pay_time_last_week','order_pay_time_last_h','order_pay_time_last_min','order_pay_time_last_s']] = raw.groupby('customer_id')['order_pay_time_last_m','order_pay_time_last_d','order_pay_time_last_week','order_pay_time_last_h','order_pay_time_last_min','order_pay_time_last_s'].last()
    t_str='2013-01-01 00:00:00'
    t=datetime.datetime.strptime(t_str, '%Y-%m-%d %H:%M:%S')
    data['goods_list_time_diff'] = data['goods_list_time_last'].map(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-t).days/364)
    data['goods_delist_time_diff'] = data['goods_delist_time_last'].map(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-t).days/364)
    data['goods_diff'] = data['goods_delist_time_diff'] - data['goods_list_time_diff']
    data['order_pay_time_last_diff'] = data['order_pay_time_last'].map(lambda x:(datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S')-t).days/364)
    ed = time.time()
    
    print(ed-st)
    
    return data

In [7]:
train_raw = raw[raw['order_pay_time'] < '2013-07-31 23:59:59']
train_raw = prerpocess(train_raw)
label_raw = set(raw[raw['order_pay_time'] > '2013-07-31 23:59:59']['customer_id'].dropna())
train_raw['labels']=train_raw.index.map(lambda x:int(x in label_raw))
test = prerpocess(raw)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:14: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  from ipykernel import kernelapp as app
C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: usi

263.07461810112
288.5027780532837


In [8]:
train_data = train_raw.drop(['goods_list_time_last', 'goods_delist_time_last', 'order_pay_time_last'], axis=1)
train_data = train_data.drop(['customer_province', 'customer_city'], axis=1)

In [9]:
catel = ['order_pay_time_last_h', 'order_pay_time_last_week', 'order_pay_time_last_m', 'order_detail_status_last', 'order_status_last', 'goods_status_last', 'goods_id_last', 'customer_gender']


In [10]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_data.drop('labels', axis=1), np.array(train_data['labels']), test_size=0.2, random_state=415)

In [11]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True)

In [12]:
y_pre = 0
for train_index , test_index in kf.split(train_data):
    X_train, X_valid, y_train, y_valid = train_data.drop('labels', axis=1).iloc[train_index], train_data.drop('labels', axis=1).iloc[test_index], np.array(train_data['labels'])[train_index], np.array(train_data['labels'])[test_index]
    import lightgbm as lgb
    param = {
    'num_leaves':121,
    'boosting_type': 'gbdt',
    'objective':'binary',
    'max_depth':7,
    'learning_rate':0.05,
    'metric':'binary_logloss'}
    trn_data = lgb.Dataset(X_train, label=y_train)
    val_data = lgb.Dataset(X_valid, label=y_valid)
    lgbm = lgb.train(param,trn_data,valid_sets=[trn_data,val_data],num_boost_round = 10000 ,early_stopping_rounds=150,verbose_eval=50, categorical_feature=catel)
    test = test[X_train.columns]
    y_pre += lgbm.predict(test)

C:\ProgramData\Anaconda3\envs\tf\lib\site-packages\lightgbm\basic.py:1295: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['customer_gender', 'goods_id_last', 'goods_status_last', 'order_detail_status_last', 'order_pay_time_last_h', 'order_pay_time_last_m', 'order_pay_time_last_week', 'order_status_last']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


Training until validation scores don't improve for 150 rounds
[50]	training's binary_logloss: 0.0675361	valid_1's binary_logloss: 0.068161
[100]	training's binary_logloss: 0.0659858	valid_1's binary_logloss: 0.0681118
[150]	training's binary_logloss: 0.064673	valid_1's binary_logloss: 0.0681932
[200]	training's binary_logloss: 0.0633046	valid_1's binary_logloss: 0.0683102
Early stopping, best iteration is:
[65]	training's binary_logloss: 0.0669495	valid_1's binary_logloss: 0.0680802
Training until validation scores don't improve for 150 rounds
[50]	training's binary_logloss: 0.0673667	valid_1's binary_logloss: 0.0693849
[100]	training's binary_logloss: 0.0658522	valid_1's binary_logloss: 0.0692658
[150]	training's binary_logloss: 0.0646497	valid_1's binary_logloss: 0.0693199
[200]	training's binary_logloss: 0.0634475	valid_1's binary_logloss: 0.069381
Early stopping, best iteration is:
[98]	training's binary_logloss: 0.0658908	valid_1's binary_logloss: 0.069265
Training until validatio

In [13]:
print(pd.DataFrame({
        'column': X_train.columns,
        'importance': lgbm.feature_importance(),
    }).sort_values(by='importance'))

                         column  importance
34          order_total_num_sum           0
4       goods_has_discount_last           1
28         is_customer_rate_sum           6
43       goods_delist_time_diff           9
5          order_total_num_last          14
13  order_detail_goods_num_last          14
2             goods_status_last          22
44                   goods_diff          22
9             order_status_last          22
36        order_pay_time_last_m          23
11        is_customer_rate_last          23
30       goods_has_discount_sum          31
10             order_count_last          31
38     order_pay_time_last_week          31
12     order_detail_status_last          37
14     order_detail_amount_last          53
35          order_total_num_ave          61
26                  goods_count          62
15    order_detail_payment_last          65
31       goods_has_discount_ave          67
27       is_customer_rate_ratio          83
29           order_detail_count 

In [14]:
y_pre = y_pre/5

In [15]:
def f(x):
    if x < 0.1:
        return 0.1
    if x > 0.9:
        return 0.9
    return x

In [16]:
test['result'] = y_pre
test['result'] = test['result'].map(f)
subm = pd.DataFrame(test['result'])
subm.to_csv('submission.csv')

In [17]:
last_test = pd.read_csv('submission.csv')
print(last_test.info)

<bound method DataFrame.info of          customer_id    result
0            1000000  0.100000
1            1000014  0.100000
2            1000034  0.100000
3            1000046  0.100000
4            1000048  0.100000
...              ...       ...
1585981      2826570  0.100000
1585982      2826571  0.100000
1585983      2826572  0.100000
1585984      2826573  0.100000
1585985      2826574  0.189466

[1585986 rows x 2 columns]>


In [18]:
print(subm.head)

<bound method NDFrame.head of                result
customer_id          
1000000      0.100000
1000014      0.100000
1000034      0.100000
1000046      0.100000
1000048      0.100000
...               ...
2826570      0.100000
2826571      0.100000
2826572      0.100000
2826573      0.100000
2826574      0.189466

[1585986 rows x 1 columns]>


In [19]:
subm.mean()

result    0.102584
dtype: float64

In [20]:
test['result'] = np.multiply(np.ones(1585986 ), 0.5)

In [21]:
subm = pd.DataFrame(test['result'])
print(subm.head)

<bound method NDFrame.head of              result
customer_id        
1000000         0.5
1000014         0.5
1000034         0.5
1000046         0.5
1000048         0.5
...             ...
2826570         0.5
2826571         0.5
2826572         0.5
2826573         0.5
2826574         0.5

[1585986 rows x 1 columns]>


In [22]:
subm.to_csv('submissionstatic1.csv')

In [23]:
test['result'] = np.multiply(np.ones(1585986 ), 0.8)
subm = pd.DataFrame(test['result'])
print(subm.head)
subm.to_csv('submissionstatic2.csv')

<bound method NDFrame.head of              result
customer_id        
1000000         0.8
1000014         0.8
1000034         0.8
1000046         0.8
1000048         0.8
...             ...
2826570         0.8
2826571         0.8
2826572         0.8
2826573         0.8
2826574         0.8

[1585986 rows x 1 columns]>
